In [1]:
!pip install image_classifiers
#!pip install keras==3.1.1 tensorflow==2.16.1
import numpy as np
import tensorflow as tf
from keras.applications import ResNet50V2
from keras.datasets import cifar100
from keras import Sequential, Input
from keras.layers import Dense, Dropout, RandomFlip, RandomTranslation, RandomRotation,RandomBrightness, RandomContrast, RandomZoom, GlobalAveragePooling2D
from keras.applications.resnet_v2 import preprocess_input
from keras.models import Model
from classification_models.keras import Classifiers
from keras.optimizers import SGD
from keras.activations import linear
from tqdm.notebook import tqdm
from keras.utils import Progbar
from keras.backend import clear_session
from tensorflow.nn import softmax_cross_entropy_with_logits
import os

2024-04-17 11:44:38.838892: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 11:44:39.762244: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
n_epoch = 50
batch_size = 64
taux_validation = 0.1
num_classes = 100
n_images = 50000 # Pour l'entrainement, et 10000 pour le test

In [3]:
validation_size = int(n_images * taux_validation)
train_size = n_images - validation_size

In [4]:
!mc cp s3/afeldmann/projet_cnam/modele_enseignant.keras /home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras
model_enseignant = Sequential([
    Input((224,224,3)),
    ResNet50V2(include_top=False, weights='imagenet', pooling="avg"),
    Dropout(0.25),
    Dense(256, activation="sigmoid", kernel_regularizer = tf.keras.regularizers.L1(0.001)),
    Dropout(0.5),
    Dense(num_classes, activation="softmax", kernel_regularizer = tf.keras.regularizers.L2(0.001))
])
# Keras 3.0 est buggé et le chargement direct ne marche pas ici, même si les poids sont bien enregistrés
model_enseignant.load_weights("/home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras")

model_enseignant.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

]11;?\mc: Configuration written to `/home/onyxia/.mc/config.json`. Please update your access credentials.
mc: Successfully created `/home/onyxia/.mc/share`.
mc: Initialized share uploads `/home/onyxia/.mc/share/uploads.json` file.
mc: Initialized share downloads `/home/onyxia/.mc/share/downloads.json` file.
...nant.keras: 135.73 MiB / 135.73 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 131.48 MiB/s 1s

2024-04-17 11:44:47.771423: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13775 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:86:00.0, compute capability: 7.5


94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [5]:
def preprocessing(image, label):
    image = tf.image.resize(image, (224, 224))
    label = tf.squeeze(tf.one_hot(label, depth = num_classes), axis = 0)
    return  image, label

augmentation_donnees_keras = Sequential([
    RandomFlip("horizontal"),
    RandomTranslation(0.2,0.2),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomContrast(0.2),
    RandomBrightness(0.2,value_range=(0,1))
])

def augmentation_donnees(image, label):
    return augmentation_donnees_keras(image/255.0, training = True)*255.0, label

def preprocess_resnet(image, label):
    return preprocess_input(image), label

def train_val_split(train_dataset, validation_size):
    X_train, y_train = train_dataset
    indices = np.random.permutation(X_train.shape[0])
    train_idx, val_idx = indices[:train_size], indices[train_size:]
    return (X_train[train_idx,...], y_train[train_idx,...]), (X_train[val_idx,...], y_train[val_idx,...])

def load_cifar_train_val():
    train_dataset, _ = cifar100.load_data()
    
    train_dataset, validation_dataset = train_val_split(train_dataset, validation_size)
    
    validation_dataset = tf.data.Dataset.from_tensor_slices(validation_dataset).map(preprocessing).batch(batch_size).map(preprocess_resnet).cache().prefetch(tf.data.AUTOTUNE)
    train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset).map(preprocessing).cache().repeat().shuffle(train_size).batch(batch_size).map(augmentation_donnees, num_parallel_calls = tf.data.AUTOTUNE).map(preprocess_resnet, num_parallel_calls = tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
    return train_dataset, validation_dataset

In [6]:
def get_modele_logits(modele):
    config = modele.layers[-1].get_config()
    config['activation'] = linear
    config['name'] = 'logits'
    res = Model(inputs=modele.inputs, outputs=[Dense(**config)(modele.layers[-2].output)])
    res.layers[-1].set_weights([x.numpy() for x in modele.layers[-1].weights])
    res.compile(metrics=['accuracy'])
    return res

@tf.function
def compte_bons(x,y):
    return tf.reduce_sum(tf.cast(tf.equal(tf.argmax(x, axis = 1), tf.argmax(y, axis = 1)), tf.float32))

@tf.function
def softmax(logits, temp):
    expo = tf.exp(logits / temp)
    return expo / tf.reduce_sum(expo, axis = 1, keepdims=True)

@tf.function
def ce(x, y_logits, temp):
    return softmax_cross_entropy_with_logits(x, y_logits / temp) * temp**2

def init_csv_log(fichier):
    with open(fichier,'w') as file:
        file.write("epoch, accuracy,val_accuracy\n")
def append_csv_log(fichier, epoch, accuracy,val_accuracy):
    with open(fichier,'a') as file:
        file.write(f"{epoch:d},{accuracy:.2f},{val_accuracy:.2f}\n")

def forward_backward_pass_impl(train_dataset_iter, etudiant_logit_model, alpha, temp, optim):
    X_batch, y_batch, enseignant_estim_softmax = next(train_dataset_iter)
    with tf.GradientTape() as tape:
        etudiant_estim_logit = etudiant_logit_model(X_batch, training = True)
        perte = alpha * softmax_cross_entropy_with_logits(y_batch,etudiant_estim_logit) + (1-alpha) * ce(enseignant_estim_softmax,etudiant_estim_logit, temp)
    grads = tape.gradient(perte, etudiant_logit_model.trainable_variables)
    optim.apply_gradients(zip(grads, etudiant_logit_model.trainable_variables))
    return compte_bons(etudiant_estim_logit,y_batch)

def distillateur_kl(etudiant, enseignant, train_dataset, validation_dataset, temp, nom_modele, n_epoch, alpha):
    etudiant_logit_model = get_modele_logits(etudiant)
    enseignant_logit_model = get_modele_logits(enseignant)
    optim = SGD(learning_rate=0.001)
    init_csv_log(f"sauvegardes/{nom_modele}_logs.csv")
    print("C'est parti pour la distillation !\n")
    val_accuracy_max = 0
    train_dataset_iter = iter(
        train_dataset
        .map(lambda images, label: (images, label, softmax(enseignant_logit_model(images, training = False), temp)), num_parallel_calls = tf.data.AUTOTUNE)
        .prefetch(tf.data.AUTOTUNE)
    )
    forward_backward_pass = tf.function(forward_backward_pass_impl)
    # La tf.function ne peut être que locale car son graphe dépend d'étudiant_logit_model et sinon Tensorflow renvoie une erreur à deux applications successives
    for epoch in range(n_epoch):
        print(f"Époque {epoch + 1} / {n_epoch}")
        n_batch = train_size//batch_size
        barre_progression = Progbar(n_batch, stateful_metrics = ["acc"])
        bons_epoque = 0
        for i in range(n_batch):
            bons_epoque += forward_backward_pass(train_dataset_iter, etudiant_logit_model, alpha, temp, optim).numpy()
            accuracy = bons_epoque / ((i+1) * batch_size)
            barre_progression.update(i + 1, values = [("acc", accuracy)])
        _, val_accuracy = etudiant.evaluate(validation_dataset, verbose = 0)
        if val_accuracy > val_accuracy_max:
            val_accuracy_max = val_accuracy
            etudiant.save(f"sauvegardes/{nom_modele}_checkpoint.keras")
        if epoch == 40:
            optim.learning_rate.assign(0.0001)
        if epoch in [10,20,50]:
            print(f"Epoque {epoch:d} - Max val accuracy {val_accuracy_max:.4f}")
        append_csv_log(f"sauvegardes/{nom_modele}_logs.csv", epoch, accuracy, val_accuracy)
        print(f"Accuracy (train) : {accuracy:.4f} | Accuracy (val) : {val_accuracy:.4f}")

In [7]:
def distillation_resnet18(temp, alpha):
    clear_session()
    train_dataset, validation_dataset = load_cifar_train_val()
    modele = new_modele_resnet()
    nom_modele =  f"model_etudiant_t{temp:d}_a{int(alpha*100):d}"
    distillateur_kl(modele, model_enseignant, train_dataset, validation_dataset, temp, nom_modele, n_epoch,0.25)
    wd = os.getcwd()
    os.system(f"cp {wd}/sauvegardes/{nom_modele}_checkpoint.keras {wd}/sauvegardes/{nom_modele}.keras")
    os.system(f"mc cp {wd}/sauvegardes/{nom_modele}.keras s3/afeldmann/projet_cnam/{nom_modele}.keras")
    os.system(f"mc cp {wd}/sauvegardes/{nom_modele}_logs.csv s3/afeldmann/projet_cnam/{nom_modele}_logs.csv")

In [8]:
def ResNet18():
    resnet18, preprocess_input = Classifiers.get('resnet18')
    resnet = resnet18((224, 224, 3), weights='imagenet', include_top=False)
    resnet_output = GlobalAveragePooling2D()(resnet.output)
    resnet = Model(inputs=resnet.input, outputs=resnet_output)
    return resnet

def new_modele_resnet():
    model = Sequential([
        Input((224,224,3)),
        ResNet18(),
        Dropout(0.25),
        Dense(256, activation="sigmoid", kernel_regularizer = tf.keras.regularizers.L1(0.001)),
        Dropout(0.5),
        Dense(num_classes, activation="softmax", kernel_regularizer = tf.keras.regularizers.L2(0.001))
    ])
    model.compile(metrics=['accuracy'])
    return model

In [9]:
distillation_resnet18(1,1) # témoin

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
44920640/44920640 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
C'est parti pour la distillation !

Époque 1 / 50


2024-04-17 11:45:43.977624: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900


703/703 ━━━━━━━━━━━━━━━━━━━━ 465s 568ms/step - acc: 0.1023


I0000 00:00:1713354814.220803    1061 service.cc:145] XLA service 0x7feaf8001cb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1713354814.221385    1061 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-04-17 11:53:34.399852: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1713354821.141912    1061 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Accuracy (train) : 0.1023 | Accuracy (val) : 0.0748
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 394s 560ms/step - acc: 0.2786
Accuracy (train) : 0.2786 | Accuracy (val) : 0.3514
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 429s 610ms/step - acc: 0.3864
Accuracy (train) : 0.3864 | Accuracy (val) : 0.4198
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 436s 620ms/step - acc: 0.4448
Accuracy (train) : 0.4448 | Accuracy (val) : 0.4672
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 446s 635ms/step - acc: 0.4883
Accuracy (train) : 0.4883 | Accuracy (val) : 0.4912
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 446s 635ms/step - acc: 0.5184
Accuracy (train) : 0.5184 | Accuracy (val) : 0.5078
Époque 7 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 448s 638ms/step - acc: 0.5432
Accuracy (train) : 0.5432 | Accuracy (val) : 0.5406
Époque 8 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 435s 619ms/step - acc: 0.5603
Accuracy (train) : 0.5603 | Accuracy (val) : 0.5668
Époque 9 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 461s 656ms/step - acc: 0.5796


2024-04-17 18:05:56.634171: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_translation_1/ReadVariableOp_2/_38}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-17 18:05:56.634519: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 16754733815599477448
2024-04-17 18:05:56.634572: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_translation_1/ReadVariableOp_2/_38}}]]
	 [[sequential_1_1/random_translation_1/add_1/_42]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-17 18:05:56.634603: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 370799861497802797
2024-04-17 18:05:56.634624: I tensorflow/core/framework/local_rendezvous.cc:422] Local ren

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t1_a100.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t1_a100.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 99.86 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t1_a100_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t1_a100_logs.csv`
Total: 669 B, Transferred: 669 B, Speed: 7.14 KiB/s


In [ ]:
distillation_resnet18(1,0.25)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 477s 646ms/step - acc: 0.1036
Accuracy (train) : 0.1036 | Accuracy (val) : 0.0928
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 414s 589ms/step - acc: 0.2773
Accuracy (train) : 0.2773 | Accuracy (val) : 0.3492
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 443s 630ms/step - acc: 0.3808
Accuracy (train) : 0.3808 | Accuracy (val) : 0.4196
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 465s 662ms/step - acc: 0.4437
Accuracy (train) : 0.4437 | Accuracy (val) : 0.4658
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 425s 605ms/step - acc: 0.4861
Accuracy (train) : 0.4861 | Accuracy (val) : 0.5056
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 445s 633ms/step - acc: 0.5125
Accuracy (train) : 0.5125 | Accuracy (val) : 0.5170
Époque 7 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 444s 632ms/step - acc: 0.5356
Accuracy (train) : 0.5356 | Accuracy (val) : 0.5384
Époque 8 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 438s 623ms/step - acc: 0.5579
Accuracy (train)

In [ ]:
distillation_resnet18(3,0.25)

In [ ]:
distillation_resnet18(8,0.25)

In [ ]:
distillation_resnet18(1,0.5)

In [ ]:
distillation_resnet18(3,0.5)

In [ ]:
distillation_resnet18(8,0.5)

In [ ]:
distillation_resnet18(1,0)

In [ ]:
distillation_resnet18(3,0)

In [ ]:
distillation_resnet18(8,0)

In [ ]:
distillation_resnet18(1,0.1)

In [ ]:
distillation_resnet18(3,0.1)

In [ ]:
distillation_resnet18(8,0.1)